## Runs DE on different clusters. Also runs DE within each cluster for each condition.
Does this over each donor and combined

In [1]:
# Input info
# se_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/SE.rds"
# sample_names = "preB,postB"
# outdir = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/de_btwnCond_RNA/minPct_0_logfc0.01"
# top_de = 3
# se_f = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/SE.rds" 
# outdir = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones"
# top_de = 3
# sample_names = "pre,post"


se_f = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/SE.rds"
cells_meta_f = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/mt_clones/cells_meta.tsv"
outdir = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/de_clone_btwnvars_RNA/"
sample_names = "pre,post"


# Parameters
ncells_thresh = 10
minPct = 0.00
logfcthresh = 0.1
test.use="wilcox"
latent.vars = "NULL"
p_thresh = 0.1 
top_de = 3
ids.reversed = T


# Parallel processing
cores = 4
# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"


In [2]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")
library(dplyr)
library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Attaching SeuratObject

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required pa

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [3]:
source(utils_script)

In [4]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
sample_names

[1] "pre"  "post"

In [5]:
dir.create(outdir)

In [6]:
cells_meta <- read.table(cells_meta_f, sep="\t", header = T, row.names="cell")
cells_meta 

#cells_meta
ids.reversed = T

if (ids.reversed){
    print('here')
    split.names <- stringr::str_split(rownames(cells_meta), "_")
    rownames(cells_meta) = lapply(split.names, function(x){paste0(x[2], "_", x[1])})
}
cells_meta = cells_meta[, order(colSums(-cells_meta))]
head(cells_meta)

,X10646T,X12295C,X13188T,X14568A,X14766T,X150T,X16126C,X16362C,X16537T,X2351C,⋯,X57C,X58C,X6093A,X6456A,X66T,X6776C,X7595A,X7775A,X8770T,X9087A
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
AAACGAAAGCGTTGCC-1_post,0,0,0,0,1,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGAAAGGCTCCTG-1_pre,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGAAAGGTTAACA-1_post,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGAAAGTGAATAC-1_pre,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGAAAGTGCTAGG-1_post,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGAAAGTGTCACT-1_pre,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGAACATAGAATG-1_post,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGAACATAGGCGA-1_pre,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGAACATCCCAAA-1_post,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


[1] "here"


,X9390G,X16223T,X1095C,X57C,X58C,X11590G,X6776C,X2442C,X12295C,X2351C,⋯,X66T,X7775A,X1949A,X8770T,X6093A,X4248C,X13108A,X15821A,X6456A,X9087A
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
post_AAACGAAAGCGTTGCC-1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAAAGGCTCCTG-1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
post_AAACGAAAGGTTAACA-1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAAAGTGAATAC-1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
post_AAACGAAAGTGCTAGG-1,1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAAAGTGTCACT-1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [7]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
se

An object of class Seurat 
97805 features across 22239 samples within 2 assays 
Active assay: RNA (19575 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

### Subset to just in cells_meta

In [8]:
se$inClones = 0
se[["inClones"]][(colnames(se) %in% row.names(cells_meta)), "inClones"] = 1

sum(se[["inClones"]])/dim(se[[]][1])

se.filt <- subset(se, subset = inClones == 1 )

[1] 6.6226e-01 1.4728e+04

In [9]:
se.filt <- AddMetaData(se.filt, cells_meta)

In [10]:
se

An object of class Seurat 
97805 features across 22239 samples within 2 assays 
Active assay: RNA (19575 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [11]:
se.filt

An object of class Seurat 
97805 features across 14728 samples within 2 assays 
Active assay: RNA (19575 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [12]:
source(utils_script)
tmp <- get.clone.sizes(se.filt)
counts.conds.norm <- tmp[["counts.norm"]]
clone.sizes <- tmp[["clone.size"]]
head(clone.sizes)

`summarise()` has grouped output by 'condition', 'donor', 'lineage'. You can override using the `.groups` argument.

Warning message in xtfrm.data.frame(x):
"cannot xtfrm data frames"
`summarise()` has grouped output by 'donor'. You can override using the `.groups` argument.



donor,lineage,norm.total,total,name,cdf.norm,cdf,index
<fct>,<fct>,<dbl>,<int>,<fct>,<dbl>,<int>,<int>
0,0,0.13454113,455,0_0,0.06727057,455,1
0,1,0.08543635,299,0_1,0.10998874,754,2
0,5,0.08101728,234,0_5,0.15049738,988,3
0,3,0.07759196,252,0_3,0.18929336,1240,4
0,2,0.07582740,264,0_2,0.22720706,1504,5
0,4,0.06708339,239,0_4,0.26074876,1743,6


In [13]:
dim(se.filt[[]])

[1] 14728    84

## plot each clone pre and post 

In [14]:
mt.vars <- colnames(cells_meta)
mt.vars

[1] "X9390G"  "X16223T" "X1095C"  "X57C"    "X58C"    "X11590G" "X6776C" 
 [8] "X2442C"  "X12295C" "X2351C"  "X13188T" "X14687G" "X9531G"  "X3847C" 
[15] "X12217G" "X2706G"  "X14945A" "X7028T"  "X14766T" "X16362C" "X64T"   
[22] "X16126C" "X16519C" "X3591A"  "X14568A" "X152C"   "X150T"   "X3244A" 
[29] "X7595A"  "X10646T" "X2623G"  "X11838A" "X16537T" "X318A"   "X66T"   
[36] "X7775A"  "X1949A"  "X8770T"  "X6093A"  "X4248C"  "X13108A" "X15821A"
[43] "X6456A"  "X9087A"

In [17]:
se.filt[[]]

,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,⋯,X66T,X7775A,X1949A,X8770T,X6093A,X4248C,X13108A,X15821A,X6456A,X9087A
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
pre_AAACGAAAGCCGCAAA-1,pre,4794,4794,38802,9366,247,213,1646,18980,8350,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAAAGGCTCCTG-1,pre,1887,1887,15412,2697,66,65,444,8994,3146,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAAAGTAGCAAT-1,pre,3068,3068,19828,5489,201,117,859,8209,4953,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAAAGTGAATAC-1,pre,1686,1686,11328,2048,58,40,407,6141,2634,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAAAGTGTCACT-1,pre,4229,4229,26680,9152,251,163,1499,7181,8434,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAACACGGCCAT-1,pre,2054,2054,16145,3572,88,91,684,8514,3196,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAACATAGGCGA-1,pre,3548,3548,34697,8294,154,194,1261,18482,6312,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAACATTAACGC-1,pre,4001,4001,19923,6293,177,104,1353,4963,7033,⋯,0,0,0,0,0,0,0,0,0,0
pre_AAACGAAGTTAACTCG-1,pre,3580,3580,40203,11082,196,187,1392,20266,7080,⋯,0,0,0,0,0,0,0,0,0,0


In [21]:
for (mt in (mt.vars)){
    print(mt)    
    expr <- FetchData(se.filt, vars = mt)
    curr.se <- se.filt[, which(expr == 1)]
    #curr.se
    p <- DimPlot(se.filt , group.by = "condition", split.by=mt)
    p2 <- DimPlot(curr.se , split.by="condition")
    curr.count = sum(se.filt[[mt]])
    ggsave(file.path(outdir, paste("count", curr.count,mt, "groupCond.png", sep="_")), p)
    ggsave(file.path(outdir, paste("count", curr.count,mt, "clrCond.png", sep="_")), p2)
}

[1] "X9390G"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X16223T"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X1095C"


...

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X6456A"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X9087A"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [22]:
outdir

[1] "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/de_clone_btwnvars_RNA/"

# Stimulus specific response in each clone

A. with variant vs without
B. top variants against top variants

In [ ]:
#    expr <- FetchData(se.filt, vars = mt)
    #curr.se <- se.filt[, which(expr == 1)]
    #curr.se

In [ ]:
# all.pvals <- data.frame(matrix(nrow=dim(se)[1],ncol=0),
#                           row.names=row.names(se))
# all.names.sig <- c()

# #sample_names
# clone.ids <- sort(unique(se$name))
# se$clone.stim <- paste(se$name, se$orig.ident, sep = "_")

# se$clone <- se$name
# Idents(se) <- "clone.stim"

# for (curr.clone in mt.vars){
#     ident.1 = paste0(curr.clone, "_", sample_names[[1]]) 
#     ident.2 = paste0(curr.clone, "_", sample_names[[2]])
#     try({
#         print(curr.clone)
#         curr.name <-paste0("clone_", curr.clone)
#         curr.clone.outdir <- file.path(btwnConds_inClone_sepDonors.outdir, paste0("clone_", curr.clone))
#         dir.create(curr.clone.outdir)

#         curr.sig <- find.markers.and.plot(se, ident.1, ident.2, curr.clone.outdir, 
#                                           curr.name=curr.clone, assay="RNA", logfcthresh=logfcthresh,
#                                           min.pct=minPct, p.thresh=p_thresh, mean.fxn=NULL, fc.name=NULL)
#         names.sig <- rownames(curr.sig)
#         all.names.sig <- c(all.names.sig, head(names.sig, top_de))
        
#         all.pvals[names.sig,curr.clone] <- curr.sig$p_val_adj_BH

#         curr_clone <- subset(se, subset = name == curr.clone)
#         avg_curr_clone <- data.frame(log1p(AverageExpression(curr_clone, verbose = FALSE)$RNA))
#         avg_curr_clone$gene <- rownames(avg_curr_clone)
#         p1 <- ggplot(avg_curr_clone, aes_string(paste0("X", curr.clone, "_", sample_names[[1]]), 
#                                                 paste0("X", curr.clone, "_", sample_names[[2]]))) + geom_point() + ggtitle(paste("Cluster", curr.clone))
#         p1 <- LabelPoints(plot = p1, points = head(names.sig,3), repel = TRUE)
#         plot_grid(p1)
#         #write.csv(response, file=file.path(curr.clone.outdir,paste0("clone_",c,".conditionDE.csv")))
#         ggsave(file.path(curr.clone.outdir,paste0("clone_",curr.clone,".allDonors.conditionScatter.png")))
        
#         })
#     }



# all.pvals[rowSums(is.na(all.pvals)) != ncol(all.pvals), ]


# ## DE sig summarize

# all.names.sig

# ### a) top 3 significant results from each comparison

# all.names.sig <- unique(all.names.sig)
# de.plots(se, all.names.sig, btwnConds_inClone_sepDonors.outdir, curr.name="allDonors.allClusters", max.size=15)

# dot <- DotPlot(se, scale=FALSE,
#                features = head(all.names.sig,20), 
#                cluster.idents=T) + RotatedAxis()
# dot
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.top3de.png"))
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.top3de.pdf"))



# ## b) averaged top p-values (only average ones that were DE)

# sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
# dot <- DotPlot(se, scale=T,
#                features = names(head(sig.all.ordered,20)), 
#                cluster.idents=T) + RotatedAxis()
# dot
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.pvalsOrdered.png"))
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.pvalsOrdered.pdf"))
# #de.plots(se, names(sig.all.ordered), outdir, curr.name="allDonors.allClusters.pvalsOrdered", max.size=20)

# sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
# dot <- DotPlot(se, scale=F,
#                features = head(names(sig.all.ordered),20), 
#                cluster.idents=T) + RotatedAxis()
# dot


In [23]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.3 LTS

Matrix products: default
BLAS/LAPACK: /home/isaac/miniconda3/envs/mttrace/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] future_1.23.0             dplyr_1.0.7              
 [3] RColorBrewer_1.1-2        metap_1.1                
 [5] cowplot_1.1.1             magrittr_2.0.1           
 [7] data.table_1.14.2         patchwork_1.1.1          
 [9] ggplot2_3.3.5             EnsDb.Hsapiens.v75_2.99.0
[11] 

In [ ]:
outdir